# Aggregating counts and rates of hospital admissions and make a table

In [10]:
import pandas as pd
import numpy as np
import functools as ft

## 0- functions and read data

In [11]:
def data_process(data, data_hosped, infection):
    #replace SMs with 5 in data and data_hosped and NaN with 0 in data_hosped 
    data = data.replace({'SM': 5})
    data_hosped = data_hosped.replace({'SM': 5, np.nan:0})
    #conver counts into int
    data[infection] = data[infection].astype(str).replace('\.0', '', regex=True).astype(int)
    data_hosped[infection] = data_hosped[infection].astype(str).replace('\.0', '', regex=True).astype(int)

    #add hosp rate
    data_hosped[infection+'_rate'] = (data_hosped[infection]/data[infection])*1000
    data_hosped[infection+'_rate'] = np.round(data_hosped[infection+'_rate'], decimals=1)

    #replace rate with SN if infection count equals 5
    data_hosped.loc[data_hosped[infection] == 5, infection+'_rate'] = 'SN'

    #calculate total using sex_female and sex_male
    total = data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_male'].index[0])][infection] + data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_female'].index[0])][infection]
    total = "{:,}".format(total)
    data_hosped['variable'] = data_hosped['variable'].str.replace('antibacterial_brit', 'total')
    
    #add comma to thousands of column count
    data_hosped[infection] = data_hosped[infection].apply(lambda x : "{:,}".format(x))
    #make new column with counts and rates
    data_hosped[infection+'_rate_count'] = data_hosped[infection].astype(str) + ' (' + data_hosped[infection+'_rate'].astype(str) + ')'
    
    #replace total (after making new column of counts and rates)
    data_hosped.at[int(data_hosped[data_hosped['variable']=='total'].index[0]),infection+'_rate_count'] = total

    #drop redundant columns and rename the needed column
    data_hosped = data_hosped.drop([infection, infection+'_rate'], axis=1)
    data_hosped = data_hosped.rename(columns={infection+'_rate_count': infection})
    #replace any row with no value (=0) with a dash 
    data_hosped[infection] = data_hosped[infection].replace({'0 (0.0)': '-'})
    #replace count and rate if rate contains SN
    data_hosped.loc[data_hosped[infection].str.contains('SN', regex=False), infection] = 'SN' 

    return data_hosped

In [12]:
summary_table_hosped_lrti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_incdt_hosped.csv')
summary_table_hosped_lrti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_incdt_hosped.csv')
summary_table_hosped_lrti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_prevl_hosped.csv')
summary_table_hosped_lrti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_prevl_hosped.csv')
summary_table_hosped_urti_combined_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_incdt_hosped.csv')
summary_table_hosped_urti_combined_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_incdt_hosped.csv')
summary_table_hosped_urti_combined_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_prevl_hosped.csv')
summary_table_hosped_urti_combined_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_prevl_hosped.csv')
summary_table_hosped_uti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_incdt_hosped.csv')
summary_table_hosped_uti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_incdt_hosped.csv')
summary_table_hosped_uti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_prevl_hosped.csv')
summary_table_hosped_uti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_prevl_hosped.csv')
summary_table_hosped_sinusitis_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_incdt_hosped.csv')
summary_table_hosped_sinusitis_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_incdt_hosped.csv')
summary_table_hosped_sinusitis_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_prevl_hosped.csv')
summary_table_hosped_sinusitis_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_prevl_hosped.csv')
summary_table_hosped_otmedia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_incdt_hosped.csv')
summary_table_hosped_otmedia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_incdt_hosped.csv')
summary_table_hosped_otmedia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_prevl_hosped.csv')
summary_table_hosped_otmedia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_prevl_hosped.csv')
summary_table_hosped_ot_externa_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_incdt_hosped.csv')
summary_table_hosped_ot_externa_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_incdt_hosped.csv')
summary_table_hosped_ot_externa_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_prevl_hosped.csv')
summary_table_hosped_ot_externa_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_prevl_hosped.csv')
summary_table_hosped_urti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_incdt_hosped.csv')
summary_table_hosped_urti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_incdt_hosped.csv')
summary_table_hosped_urti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_prevl_hosped.csv')
summary_table_hosped_urti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_prevl_hosped.csv')
summary_table_hosped_cough_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_incdt_hosped.csv')
summary_table_hosped_cough_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_incdt_hosped.csv')
summary_table_hosped_cough_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_prevl_hosped.csv')
summary_table_hosped_cough_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_prevl_hosped.csv')
summary_table_hosped_cough_cold_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_incdt_hosped.csv')
summary_table_hosped_cough_cold_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_incdt_hosped.csv')
summary_table_hosped_cough_cold_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_prevl_hosped.csv')
summary_table_hosped_cough_cold_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_prevl_hosped.csv')
summary_table_hosped_throat_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_incdt_hosped.csv')
summary_table_hosped_throat_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_incdt_hosped.csv')
summary_table_hosped_throat_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_prevl_hosped.csv')
summary_table_hosped_throat_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_prevl_hosped.csv')

In [13]:
summary_table_lrti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_incdt.csv')
summary_table_lrti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_incdt.csv')
summary_table_lrti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_no_abs_prevl.csv')
summary_table_lrti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_abs_prevl.csv')
summary_table_urti_combined_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_incdt.csv')
summary_table_urti_combined_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_incdt.csv')
summary_table_urti_combined_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_no_abs_prevl.csv')
summary_table_urti_combined_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_abs_prevl.csv')
summary_table_uti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_incdt.csv')
summary_table_uti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_incdt.csv')
summary_table_uti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_no_abs_prevl.csv')
summary_table_uti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_abs_prevl.csv')
summary_table_sinusitis_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_incdt.csv')
summary_table_sinusitis_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_incdt.csv')
summary_table_sinusitis_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_no_abs_prevl.csv')
summary_table_sinusitis_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_abs_prevl.csv')
summary_table_otmedia_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_incdt.csv')
summary_table_otmedia_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_incdt.csv')
summary_table_otmedia_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_no_abs_prevl.csv')
summary_table_otmedia_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_abs_prevl.csv')
summary_table_ot_externa_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_incdt.csv')
summary_table_ot_externa_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_incdt.csv')
summary_table_ot_externa_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_no_abs_prevl.csv')
summary_table_ot_externa_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_abs_prevl.csv')
summary_table_urti_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_incdt.csv')
summary_table_urti_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_incdt.csv')
summary_table_urti_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_no_abs_prevl.csv')
summary_table_urti_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_abs_prevl.csv')
summary_table_cough_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_incdt.csv')
summary_table_cough_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_incdt.csv')
summary_table_cough_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_no_abs_prevl.csv')
summary_table_cough_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_abs_prevl.csv')
summary_table_cough_cold_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_incdt.csv')
summary_table_cough_cold_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_incdt.csv')
summary_table_cough_cold_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_no_abs_prevl.csv')
summary_table_cough_cold_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_abs_prevl.csv')
summary_table_throat_no_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_incdt.csv')
summary_table_throat_abs_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_incdt.csv')
summary_table_throat_no_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_no_abs_prevl.csv')
summary_table_throat_abs_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_abs_prevl.csv')

In [14]:
#select relevant columns of each df
summary_table_lrti_no_abs_incdt = summary_table_lrti_no_abs_incdt[['variable', 'count']]
summary_table_lrti_abs_incdt = summary_table_lrti_abs_incdt[['variable', 'count']]
summary_table_lrti_no_abs_prevl = summary_table_lrti_no_abs_prevl[['variable', 'count']]
summary_table_lrti_abs_prevl = summary_table_lrti_abs_prevl[['variable', 'count']]
summary_table_urti_combined_no_abs_incdt = summary_table_urti_combined_no_abs_incdt[['variable', 'count']]
summary_table_urti_combined_abs_incdt = summary_table_urti_combined_abs_incdt[['variable', 'count']]
summary_table_urti_combined_no_abs_prevl = summary_table_urti_combined_no_abs_prevl[['variable', 'count']]
summary_table_urti_combined_abs_prevl = summary_table_urti_combined_abs_prevl[['variable', 'count']]
summary_table_uti_no_abs_incdt = summary_table_uti_no_abs_incdt[['variable', 'count']]
summary_table_uti_abs_incdt = summary_table_uti_abs_incdt[['variable', 'count']]
summary_table_uti_no_abs_prevl = summary_table_uti_no_abs_prevl[['variable', 'count']]
summary_table_uti_abs_prevl = summary_table_uti_abs_prevl[['variable', 'count']]
summary_table_sinusitis_no_abs_incdt = summary_table_sinusitis_no_abs_incdt[['variable', 'count']]
summary_table_sinusitis_abs_incdt = summary_table_sinusitis_abs_incdt[['variable', 'count']]
summary_table_sinusitis_no_abs_prevl = summary_table_sinusitis_no_abs_prevl[['variable', 'count']]
summary_table_sinusitis_abs_prevl = summary_table_sinusitis_abs_prevl[['variable', 'count']]
summary_table_otmedia_no_abs_incdt = summary_table_otmedia_no_abs_incdt[['variable', 'count']]
summary_table_otmedia_abs_incdt = summary_table_otmedia_abs_incdt[['variable', 'count']]
summary_table_otmedia_no_abs_prevl = summary_table_otmedia_no_abs_prevl[['variable', 'count']]
summary_table_otmedia_abs_prevl = summary_table_otmedia_abs_prevl[['variable', 'count']]
summary_table_ot_externa_no_abs_incdt = summary_table_ot_externa_no_abs_incdt[['variable', 'count']]
summary_table_ot_externa_abs_incdt = summary_table_ot_externa_abs_incdt[['variable', 'count']]
summary_table_ot_externa_no_abs_prevl = summary_table_ot_externa_no_abs_prevl[['variable', 'count']]
summary_table_ot_externa_abs_prevl = summary_table_ot_externa_abs_prevl[['variable', 'count']]
summary_table_urti_no_abs_incdt = summary_table_urti_no_abs_incdt[['variable', 'count']]
summary_table_urti_abs_incdt = summary_table_urti_abs_incdt[['variable', 'count']]
summary_table_urti_no_abs_prevl = summary_table_urti_no_abs_prevl[['variable', 'count']]
summary_table_urti_abs_prevl = summary_table_urti_abs_prevl[['variable', 'count']]
summary_table_cough_no_abs_incdt = summary_table_cough_no_abs_incdt[['variable', 'count']]
summary_table_cough_abs_incdt = summary_table_cough_abs_incdt[['variable', 'count']]
summary_table_cough_no_abs_prevl = summary_table_cough_no_abs_prevl[['variable', 'count']]
summary_table_cough_abs_prevl = summary_table_cough_abs_prevl[['variable', 'count']]
summary_table_cough_cold_no_abs_incdt = summary_table_cough_cold_no_abs_incdt[['variable', 'count']]
summary_table_cough_cold_abs_incdt = summary_table_cough_cold_abs_incdt[['variable', 'count']]
summary_table_cough_cold_no_abs_prevl = summary_table_cough_cold_no_abs_prevl[['variable', 'count']]
summary_table_cough_cold_abs_prevl = summary_table_cough_cold_abs_prevl[['variable', 'count']]
summary_table_throat_no_abs_incdt = summary_table_throat_no_abs_incdt[['variable', 'count']]
summary_table_throat_abs_incdt = summary_table_throat_abs_incdt[['variable', 'count']]
summary_table_throat_no_abs_prevl = summary_table_throat_no_abs_prevl[['variable', 'count']]
summary_table_throat_abs_prevl = summary_table_throat_abs_prevl[['variable', 'count']]

In [15]:
#merge baseline dfs
dfs_tab = [summary_table_lrti_no_abs_incdt, summary_table_lrti_abs_incdt, summary_table_lrti_no_abs_prevl, summary_table_lrti_abs_prevl, 
        summary_table_urti_combined_no_abs_incdt, summary_table_urti_combined_abs_incdt, summary_table_urti_combined_no_abs_prevl, summary_table_urti_combined_abs_prevl,
        summary_table_uti_no_abs_incdt, summary_table_uti_abs_incdt, summary_table_uti_no_abs_prevl, summary_table_uti_abs_prevl,
        summary_table_sinusitis_no_abs_incdt, summary_table_sinusitis_abs_incdt, summary_table_sinusitis_no_abs_prevl, summary_table_sinusitis_abs_prevl,
        summary_table_otmedia_no_abs_incdt, summary_table_otmedia_abs_incdt, summary_table_otmedia_no_abs_prevl, summary_table_otmedia_abs_prevl,
        summary_table_ot_externa_no_abs_incdt, summary_table_ot_externa_abs_incdt, summary_table_ot_externa_no_abs_prevl, summary_table_ot_externa_abs_prevl,
        summary_table_urti_no_abs_incdt, summary_table_urti_abs_incdt, summary_table_urti_no_abs_prevl, summary_table_urti_abs_prevl,
        summary_table_cough_no_abs_incdt, summary_table_cough_abs_incdt, summary_table_cough_no_abs_prevl, summary_table_cough_abs_prevl,
        summary_table_cough_cold_no_abs_incdt, summary_table_cough_cold_abs_incdt, summary_table_cough_cold_no_abs_prevl, summary_table_cough_cold_abs_prevl,
        summary_table_throat_no_abs_incdt, summary_table_throat_abs_incdt, summary_table_throat_no_abs_prevl, summary_table_throat_abs_prevl
        ]

df_final_tab = ft.reduce(lambda left, right: pd.merge(left, right, on='variable', how='outer'), dfs_tab)

df_final_tab.columns = ['variable', 'lrti_no_abs_incdt', 'lrti_abs_incdt', 'lrti_no_abs_prevl', 'lrti_abs_prevl', 
        'urti_combined_no_abs_incdt', 'urti_combined_abs_incdt', 'urti_combined_no_abs_prevl', 'urti_combined_abs_prevl',
        'uti_no_abs_incdt', 'uti_abs_incdt', 'uti_no_abs_prevl', 'uti_abs_prevl',
        'sinusitis_no_abs_incdt', 'sinusitis_abs_incdt', 'sinusitis_no_abs_prevl', 'sinusitis_abs_prevl',
        'otmedia_no_abs_incdt', 'otmedia_abs_incdt', 'otmedia_no_abs_prevl', 'otmedia_abs_prevl',
        'ot_externa_no_abs_incdt', 'ot_externa_abs_incdt', 'ot_externa_no_abs_prevl', 'ot_externa_abs_prevl',
        'urti_no_abs_incdt', 'urti_abs_incdt', 'urti_no_abs_prevl', 'urti_abs_prevl',
        'cough_no_abs_incdt', 'cough_abs_incdt', 'cough_no_abs_prevl', 'cough_abs_prevl',
        'cough_cold_no_abs_incdt', 'cough_cold_abs_incdt', 'cough_cold_no_abs_prevl', 'cough_cold_abs_prevl',
        'throat_no_abs_incdt', 'throat_abs_incdt', 'throat_no_abs_prevl', 'throat_abs_prevl'
        ]

#custom sort baseline dfs
df_final_tab['variable'] = pd.Categorical(df_final_tab['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
                                                                    'sex_female', 'sex_male',
                                                                    'CCI_cat_very_low', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_high', 'CCI_cat_very_high', 
                                                                    'flu_vaccine_no', 'flu_vaccine_yes',
                                                                    'bmi_cat_underweight', 'bmi_cat_healthy_weight', 'bmi_cat_overweight', 'bmi_cat_obese', 'bmi_cat_unknown',
                                                                    'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
                                                                    'imd_very_affluent', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_very_unaffluent', 'imd_unknown',
                                                                    'ethnicity_non-white', 'ethnicity_unknown', 'ethnicity_white',
                                                                    'smoking_status_ex_smoker', 'smoking_status_never_smoked', 'smoking_status_smoker', 'smoking_status_unknown',
                                                                    'season_autumn', 'season_spring', 'season_summer', 'season_winter',
                                                                    'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic',
                                                                    'antibacterial_brit'])
df_final_tab = df_final_tab.sort_values("variable").reset_index(drop=True)

In [16]:
#select relevant columns of each df
summary_table_hosped_lrti_no_abs_incdt = summary_table_hosped_lrti_no_abs_incdt[['variable', 'count']]
summary_table_hosped_lrti_abs_incdt = summary_table_hosped_lrti_abs_incdt[['variable', 'count']]
summary_table_hosped_lrti_no_abs_prevl = summary_table_hosped_lrti_no_abs_prevl[['variable', 'count']]
summary_table_hosped_lrti_abs_prevl = summary_table_hosped_lrti_abs_prevl[['variable', 'count']]
summary_table_hosped_urti_combined_no_abs_incdt = summary_table_hosped_urti_combined_no_abs_incdt[['variable', 'count']]
summary_table_hosped_urti_combined_abs_incdt = summary_table_hosped_urti_combined_abs_incdt[['variable', 'count']]
summary_table_hosped_urti_combined_no_abs_prevl = summary_table_hosped_urti_combined_no_abs_prevl[['variable', 'count']]
summary_table_hosped_urti_combined_abs_prevl = summary_table_hosped_urti_combined_abs_prevl[['variable', 'count']]
summary_table_hosped_uti_no_abs_incdt = summary_table_hosped_uti_no_abs_incdt[['variable', 'count']]
summary_table_hosped_uti_abs_incdt = summary_table_hosped_uti_abs_incdt[['variable', 'count']]
summary_table_hosped_uti_no_abs_prevl = summary_table_hosped_uti_no_abs_prevl[['variable', 'count']]
summary_table_hosped_uti_abs_prevl = summary_table_hosped_uti_abs_prevl[['variable', 'count']]
summary_table_hosped_sinusitis_no_abs_incdt = summary_table_hosped_sinusitis_no_abs_incdt[['variable', 'count']]
summary_table_hosped_sinusitis_abs_incdt = summary_table_hosped_sinusitis_abs_incdt[['variable', 'count']]
summary_table_hosped_sinusitis_no_abs_prevl = summary_table_hosped_sinusitis_no_abs_prevl[['variable', 'count']]
summary_table_hosped_sinusitis_abs_prevl = summary_table_hosped_sinusitis_abs_prevl[['variable', 'count']]
summary_table_hosped_otmedia_no_abs_incdt = summary_table_hosped_otmedia_no_abs_incdt[['variable', 'count']]
summary_table_hosped_otmedia_abs_incdt = summary_table_hosped_otmedia_abs_incdt[['variable', 'count']]
summary_table_hosped_otmedia_no_abs_prevl = summary_table_hosped_otmedia_no_abs_prevl[['variable', 'count']]
summary_table_hosped_otmedia_abs_prevl = summary_table_hosped_otmedia_abs_prevl[['variable', 'count']]
summary_table_hosped_ot_externa_no_abs_incdt = summary_table_hosped_ot_externa_no_abs_incdt[['variable', 'count']]
summary_table_hosped_ot_externa_abs_incdt = summary_table_hosped_ot_externa_abs_incdt[['variable', 'count']]
summary_table_hosped_ot_externa_no_abs_prevl = summary_table_hosped_ot_externa_no_abs_prevl[['variable', 'count']]
summary_table_hosped_ot_externa_abs_prevl = summary_table_hosped_ot_externa_abs_prevl[['variable', 'count']]
summary_table_hosped_urti_no_abs_incdt = summary_table_hosped_urti_no_abs_incdt[['variable', 'count']]
summary_table_hosped_urti_abs_incdt = summary_table_hosped_urti_abs_incdt[['variable', 'count']]
summary_table_hosped_urti_no_abs_prevl = summary_table_hosped_urti_no_abs_prevl[['variable', 'count']]
summary_table_hosped_urti_abs_prevl = summary_table_hosped_urti_abs_prevl[['variable', 'count']]
summary_table_hosped_cough_no_abs_incdt = summary_table_hosped_cough_no_abs_incdt[['variable', 'count']]
summary_table_hosped_cough_abs_incdt = summary_table_hosped_cough_abs_incdt[['variable', 'count']]
summary_table_hosped_cough_no_abs_prevl = summary_table_hosped_cough_no_abs_prevl[['variable', 'count']]
summary_table_hosped_cough_abs_prevl = summary_table_hosped_cough_abs_prevl[['variable', 'count']]
summary_table_hosped_cough_cold_no_abs_incdt = summary_table_hosped_cough_cold_no_abs_incdt[['variable', 'count']]
summary_table_hosped_cough_cold_abs_incdt = summary_table_hosped_cough_cold_abs_incdt[['variable', 'count']]
summary_table_hosped_cough_cold_no_abs_prevl = summary_table_hosped_cough_cold_no_abs_prevl[['variable', 'count']]
summary_table_hosped_cough_cold_abs_prevl = summary_table_hosped_cough_cold_abs_prevl[['variable', 'count']]
summary_table_hosped_throat_no_abs_incdt = summary_table_hosped_throat_no_abs_incdt[['variable', 'count']]
summary_table_hosped_throat_abs_incdt = summary_table_hosped_throat_abs_incdt[['variable', 'count']]
summary_table_hosped_throat_no_abs_prevl = summary_table_hosped_throat_no_abs_prevl[['variable', 'count']]
summary_table_hosped_throat_abs_prevl = summary_table_hosped_throat_abs_prevl[['variable', 'count']]

In [17]:
#merge hosped dfs
dfs_tab_hosped = [summary_table_hosped_lrti_no_abs_incdt, summary_table_hosped_lrti_abs_incdt, summary_table_hosped_lrti_no_abs_prevl, summary_table_hosped_lrti_abs_prevl, 
        summary_table_hosped_urti_combined_no_abs_incdt, summary_table_hosped_urti_combined_abs_incdt, summary_table_hosped_urti_combined_no_abs_prevl, summary_table_hosped_urti_combined_abs_prevl,
        summary_table_hosped_uti_no_abs_incdt, summary_table_hosped_uti_abs_incdt, summary_table_hosped_uti_no_abs_prevl, summary_table_hosped_uti_abs_prevl,
        summary_table_hosped_sinusitis_no_abs_incdt, summary_table_hosped_sinusitis_abs_incdt, summary_table_hosped_sinusitis_no_abs_prevl, summary_table_hosped_sinusitis_abs_prevl,
        summary_table_hosped_otmedia_no_abs_incdt, summary_table_hosped_otmedia_abs_incdt, summary_table_hosped_otmedia_no_abs_prevl, summary_table_hosped_otmedia_abs_prevl,
        summary_table_hosped_ot_externa_no_abs_incdt, summary_table_hosped_ot_externa_abs_incdt, summary_table_hosped_ot_externa_no_abs_prevl, summary_table_hosped_ot_externa_abs_prevl,
        summary_table_hosped_urti_no_abs_incdt, summary_table_hosped_urti_abs_incdt, summary_table_hosped_urti_no_abs_prevl, summary_table_hosped_urti_abs_prevl,
        summary_table_hosped_cough_no_abs_incdt, summary_table_hosped_cough_abs_incdt, summary_table_hosped_cough_no_abs_prevl, summary_table_hosped_cough_abs_prevl,
        summary_table_hosped_cough_cold_no_abs_incdt, summary_table_hosped_cough_cold_abs_incdt, summary_table_hosped_cough_cold_no_abs_prevl, summary_table_hosped_cough_cold_abs_prevl,
        summary_table_hosped_throat_no_abs_incdt, summary_table_hosped_throat_abs_incdt, summary_table_hosped_throat_no_abs_prevl, summary_table_hosped_throat_abs_prevl
        ]

df_final_tab_hosped = ft.reduce(lambda left, right: pd.merge(left, right, on='variable', how='outer'), dfs_tab_hosped)

df_final_tab_hosped.columns = ['variable', 'lrti_no_abs_incdt', 'lrti_abs_incdt', 'lrti_no_abs_prevl', 'lrti_abs_prevl', 
        'urti_combined_no_abs_incdt', 'urti_combined_abs_incdt', 'urti_combined_no_abs_prevl', 'urti_combined_abs_prevl',
        'uti_no_abs_incdt', 'uti_abs_incdt', 'uti_no_abs_prevl', 'uti_abs_prevl',
        'sinusitis_no_abs_incdt', 'sinusitis_abs_incdt', 'sinusitis_no_abs_prevl', 'sinusitis_abs_prevl',
        'otmedia_no_abs_incdt', 'otmedia_abs_incdt', 'otmedia_no_abs_prevl', 'otmedia_abs_prevl',
        'ot_externa_no_abs_incdt', 'ot_externa_abs_incdt', 'ot_externa_no_abs_prevl', 'ot_externa_abs_prevl',
        'urti_no_abs_incdt', 'urti_abs_incdt', 'urti_no_abs_prevl', 'urti_abs_prevl',
        'cough_no_abs_incdt', 'cough_abs_incdt', 'cough_no_abs_prevl', 'cough_abs_prevl',
        'cough_cold_no_abs_incdt', 'cough_cold_abs_incdt', 'cough_cold_no_abs_prevl', 'cough_cold_abs_prevl',
        'throat_no_abs_incdt', 'throat_abs_incdt', 'throat_no_abs_prevl', 'throat_abs_prevl'
        ]

#custom sort factors of variables column 
df_final_tab_hosped['variable'] = pd.Categorical(df_final_tab_hosped['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
                                                                    'sex_female', 'sex_male',
                                                                    'CCI_cat_very_low', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_high', 'CCI_cat_very_high', 
                                                                    'flu_vaccine_no', 'flu_vaccine_yes',
                                                                    'bmi_cat_underweight', 'bmi_cat_healthy_weight', 'bmi_cat_overweight', 'bmi_cat_obese', 'bmi_cat_unknown',
                                                                    'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
                                                                    'imd_very_affluent', 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_very_unaffluent', 'imd_unknown',
                                                                    'ethnicity_non-white', 'ethnicity_unknown', 'ethnicity_white',
                                                                    'smoking_status_ex_smoker', 'smoking_status_never_smoked', 'smoking_status_smoker', 'smoking_status_unknown',
                                                                    'season_autumn', 'season_spring', 'season_summer', 'season_winter',
                                                                    'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic', 
                                                                    'antibacterial_brit'])
df_final_tab_hosped = df_final_tab_hosped.sort_values("variable").reset_index(drop=True)

## 1- process data to make a table of counts and rates, and save it

In [19]:
data_hosped = data_process(df_final_tab, df_final_tab_hosped, 'lrti_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'lrti_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'lrti_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'lrti_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_no_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_abs_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_no_abs_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_abs_prevl')

#save hosped count and rate table
data_hosped.to_csv(f'../output/aggregate_tables/hosped_count_rate_table_all_infections.csv', index=False)